In [ ]:
import os
import sys
from pathlib import Path
from loguru import logger

from monai_training.preprocess import DataSetProcesser
from monai_training import training
from mri_data import file_manager as fm


In [6]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [ ]:
drive_root = fm.get_drive_root()
projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
dataroot = drive_root / "3Tpioneer_bids"

dataset_name = "single-vs-multi1"

In [8]:
subjects = [
    1010,
    1011,
    1019,
    1029,
    1033,
    1037,
    1038,
    1080,
    1085,
    1087,
    1089,
    1109,
    1119,
    1152,
    1163,
    1188,
    1191,
    1234,
    1259,
    1265,
    1272,
    1280,
    1293,
    1321,
    1355,
    1437,
    1498,
    1518,
    1540,
    1547,
    1548,
    2081,
    2083,
    2097,
    2126,
    2132,
    2144,
    2146,
    2164,
    2187]

In [9]:
def has_subject(scan, subjects: list[int]) -> bool:
    if int(scan.subid) in subjects:
        return True
    else:
        return False

In [ ]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset_proc.filter([has_subject], [(subjects,)])
dataset = dataset_proc.dataset
dataset.sort()

In [ ]:
from monai_training import preprocess


dataset = training.assign_conditions(dataset, 0.2)
preprocess.save_dataset(dataset, "dataset.json")